# しりとりモデルの作成
## 前準備

In [ ]:
!pip install -q wordfreq fugashi[unidic-lite] trl

In [ ]:
from wordfreq import top_n_list
import fugashi, itertools




In [ ]:
import re

ascii_pat = re.compile(r'^[A-Za-z0-9_]+$')   # ← 完全 ASCII を検出
tagger = fugashi.Tagger()
good_pos = {"名詞", "形容詞", "動詞"}

def is_good(word):
    # ① ASCII なら除外
    if ascii_pat.fullmatch(word):
        return False
    # ② 品詞チェック（名詞・形容詞のみ採用）
    return tagger(word)[0].feature.pos1 in good_pos

raw = [w for w in top_n_list("ja", 10_000) if w.isalpha() and len(w) >= 3]
VOCAB = [w for w in raw if is_good(w)]

In [ ]:
import random
from datasets import Dataset


kana_pat = re.compile(r'[ぁ-ゖァ-ヺー]')  # ひらがな・カタカナ・長音符を許可

def build_pairs(vocab, size=50_000):
    pairs = []
    for _ in range(size):
        w1 = random.choice(vocab)
        tail = w1[-1]

        # w2候補：w1の末尾と一致する先頭文字かつ、最後が「ん」でなく、先頭がかな文字
        cand = [
            w for w in vocab
            if w[0] == tail and w[-1] != "ん" and kana_pat.fullmatch(w[0])
        ]

        if not cand:
            continue

        w2 = random.choice(cand)
        pairs.append({"prompt": f"{w1} → ", "completion": w2})

    return Dataset.from_list(pairs)


dataset = build_pairs(VOCAB)


In [ ]:
print(dataset[1])

## 報酬の定義

In [ ]:
import re, math, fugashi
import numpy as np
tagger = fugashi.Tagger()
kana_pat = re.compile('[ぁ-ゔー]')




# しりとりのルールに従って報酬を計算する関数
def shiritori_reward(prompts, completions, **kw):
    rewards = []

    # 各プロンプトと応答のペアでループ
    for p, c in zip(prompts, completions):
        # 応答の最初の3文字以内から、最初に出てくるかな文字を取得
        head = c[0] if len(c)>0 else ""
        # プロンプト（prompts）の最後のかな文字を取得
        tail = p[-1] if len(p)>0 else ""
        
        # 基本点：しりとりがつながっていれば +3.0、つながっていなければ -10.0
        ok = 3.0 if head and head == tail else -10.0

        # ペナルティ：応答の最後の文字が「ん」なら -1.0（しりとり終了）
        penalty = -3.0 if c[-1] == 'ん' else 0.0

        # 長さの制約

        length = 5.0 if len(c)>2 and len(c)<10 else -5.0

        # 矢印の出力

        arrow = -5.0 if "→" in c else 5.0

        # 語彙外チェック：語彙リスト（VOCAB）に存在しない単語なら +4.0
        oov = 4.0 if c not in VOCAB else 0.0

        # 合計スコアを記録
        rewards.append(ok + penalty + oov + arrow+length)

    # 各ペアに対する報酬のリストを返す
    return rewards


In [ ]:
from trl import GRPOConfig, GRPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# https://huggingface.co/google/gemma-3-1b-it

model_name = "google/gemma-3-1b-it"

model = AutoModelForCausalLM.from_pretrained(model_name,  device_map={"": 0} )
tokenizer   = AutoTokenizer.from_pretrained(model_name)



In [ ]:
# 500個取得

sample_dataset = dataset.shuffle(seed=42).select(range(0,300))

In [ ]:
sample_dataset


In [ ]:
grpo_cfg = GRPOConfig(
    num_generations=8,          # 1 プロンプトにつき回答 8 本
    temperature=0.8,
    max_prompt_length=32,
    max_completion_length=32,
    logging_strategy="epoch",
    num_train_epochs    = 5,
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=sample_dataset,
    reward_funcs=[shiritori_reward],
    args=grpo_cfg,
)

In [ ]:
trainer.train()


In [ ]:
SAVE_DIR = "./shiritori-gemma-model"
trainer.save_model(SAVE_DIR)                # モデルの保存
tokenizer.save_pretrained(SAVE_DIR)              # トークナイザも忘れずに